In [ ]:
# Overall step of getting OD Cost Matrix

### First, all hospitals has to have population density fields - spatial join



### 1. Get driving time from PHC to Centroid point
### 2. Get weighted population by driving time and PopDen
### 3. Dissolve by ID and sum the weighted population
### 4. Join the field to PHC point

### 5. Get driving time from Centroid point to PHC point
### 6. Get weighted provider to population ratio by driving time and PopDen
### 7. Dissolve by ID and sum population ratio

# First prepare Census 2015 Centroid point data with population density

In [4]:
import arcpy
from arcpy import env
try:
    env.workspace = "C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/HealthPlaceNew.gdb"

    ### Calculate the size of each polygon (Residential area)
    # Set local variables
    inTable = "Census2015ResAreaJP"
    fieldName1 = "AreaRes"
    fieldName2 = "PopDen"
    expression1 = "!shape.area@SQUAREKILOMETERS!"
    expression2 = "!TOTPOP_H27!/!AreaRes!"
    # Execute AddField
    arcpy.AddField_management(inTable, fieldName1, "DOUBLE")
    arcpy.AddField_management(inTable, fieldName2, "DOUBLE")
    # Execute CalculateField - Get a area size in square kilometers
    arcpy.CalculateField_management(inTable, fieldName1, expression1, "PYTHON_9.3")
    arcpy.CalculateField_management(inTable, fieldName2, expression2, "PYTHON_9.3")

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

In [5]:
import arcpy
from arcpy import env
try:
    env.workspace = "C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/HealthPlaceNew.gdb"
    
    # Create centroid point feature
    arcpy.FeatureToPoint_management("Census2015ResAreaJP", "C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/OD_Matrix.gdb/Census2015Res_CentroidJP", "CENTROID")

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [ ]:
# Create the chou-chou boundary polygon feature with Population density

In [22]:
import arcpy
from arcpy import env
try:
    env.workspace = "C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/OD_Matrix.gdb"
    ### First, delete all unnecesarry fields and create a census polygon data
    # list all field name
    field_names = [f.name for f in arcpy.ListFields("Census2015PopDenChugoku")]
    print(field_names)
    # Choose the fields want to be deleted
    delfield = ['Census2015PopChugoku_KEN_NAME', 'Census2015PopChugoku_SHICHO_NAME', 'Census2015PopChugoku_SHI_NAME', 'Census2015PopChugoku_KU_NAME', 'Census2015PopChugoku_GUN_NAME', 'Census2015PopChugoku_CHOSON_NAME', 'Census2015PopChugoku_OAZA_NAME', 'Census2015PopChugoku_KOAZA_NAME', 'Census2015PopChugoku_SUM_AREA', 'Census2015PopChugoku_TOTPOP_H27', 'Census2015Res_CentroidChugoku_OBJECTID', 'Census2015Res_CentroidChugoku_KEN_NAME', 'Census2015Res_CentroidChugoku_SHICHO_NAME', 'Census2015Res_CentroidChugoku_SHI_NAME', 'Census2015Res_CentroidChugoku_KU_NAME', 'Census2015Res_CentroidChugoku_GUN_NAME', 'Census2015Res_CentroidChugoku_CHOSON_NAME', 'Census2015Res_CentroidChugoku_OAZA_NAME', 'Census2015Res_CentroidChugoku_KOAZA_NAME', 'Census2015Res_CentroidChugoku_SUM_AREA', 'Census2015Res_CentroidChugoku_TOTPOP_H27', 'Census2015Res_CentroidChugoku_AreaRes','Census2015Res_CentroidChugoku_ORIG_FID']    # Delete fields
    arcpy.DeleteField_management('Census2015PopDenChugoku',delfield)

except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

['OBJECTID', 'Shape', 'Census2015PopChugoku_KEN_NAME', 'Census2015PopChugoku_SHICHO_NAME', 'Census2015PopChugoku_SHI_NAME', 'Census2015PopChugoku_KU_NAME', 'Census2015PopChugoku_GUN_NAME', 'Census2015PopChugoku_CHOSON_NAME', 'Census2015PopChugoku_OAZA_NAME', 'Census2015PopChugoku_KOAZA_NAME', 'Census2015PopChugoku_SUM_AREA', 'Census2015PopChugoku_TOTPOP_H27', 'Census2015Res_CentroidChugoku_OBJECTID', 'Census2015Res_CentroidChugoku_KEN_NAME', 'Census2015Res_CentroidChugoku_SHICHO_NAME', 'Census2015Res_CentroidChugoku_SHI_NAME', 'Census2015Res_CentroidChugoku_KU_NAME', 'Census2015Res_CentroidChugoku_GUN_NAME', 'Census2015Res_CentroidChugoku_CHOSON_NAME', 'Census2015Res_CentroidChugoku_OAZA_NAME', 'Census2015Res_CentroidChugoku_KOAZA_NAME', 'Census2015Res_CentroidChugoku_SUM_AREA', 'Census2015Res_CentroidChugoku_TOTPOP_H27', 'Census2015Res_CentroidChugoku_AreaRes', 'Census2015Res_CentroidChugoku_PopDen', 'Census2015Res_CentroidChugoku_ORIG_FID', 'Shape_Length', 'Shape_Area']


# Second, Spatial join each hospitals with census area data to attach population density

In [23]:
### Import system modules
import arcpy
import os

try:
    
    # Set local variables
    inWorkspace1 = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\HealthPlaceNew.gdb"
    inWorkspace2 = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\OD_Matrix.gdb"
    outWorkdspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\OD_Matrix.gdb"
    targetFeatures = os.path.join(inWorkspace1, "PHCNaika_Chugoku")
    joinFeatures = os.path.join(inWorkspace2, "Census2015PopDenChugoku")
    
    # Output will be the target features: PHCCover Polygon with a sum of population field 
    outfc = os.path.join(outWorkdspace, "PHC_NaikaChugoku_PopDen")
    
    # Create a new fieldmappings and add the two input feature classes.
    fieldmappings = arcpy.FieldMappings()
    fieldmappings.addTable(targetFeatures)
    fieldmappings.addTable(joinFeatures)
       
    # Delete fields that are no longer applicable, such as city CITY_NAME and CITY_FIPS
    # as only the first value will be used by default
    x = fieldmappings.findFieldMapIndex("P04_001")
    fieldmappings.removeFieldMap(x)
    y = fieldmappings.findFieldMapIndex("P04_002")
    fieldmappings.removeFieldMap(y)
    z = fieldmappings.findFieldMapIndex("P04_003")
    fieldmappings.removeFieldMap(z)
    c = fieldmappings.findFieldMapIndex("P04_004")
    fieldmappings.removeFieldMap(c)
    d = fieldmappings.findFieldMapIndex("P04_005")
    fieldmappings.removeFieldMap(d)
    e = fieldmappings.findFieldMapIndex("P04_006")
    fieldmappings.removeFieldMap(e)
    f = fieldmappings.findFieldMapIndex("P04_007")
    fieldmappings.removeFieldMap(f)
    h = fieldmappings.findFieldMapIndex("Naika")
    fieldmappings.removeFieldMap(h)

    #Run the Spatial Join tool, using the defaults for the join operation and join type
    arcpy.SpatialJoin_analysis(targetFeatures, joinFeatures, outfc, "JOIN_ONE_TO_ONE", "KEEP_ALL", fieldmappings,"COMPLETELY_WITHIN")

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [24]:
### Import system modules
import arcpy
import os

try:
    
    # Set local variables
    inWorkspace1 = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\HealthPlaceNew.gdb"
    inWorkspace2 = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\OD_Matrix.gdb"
    outWorkdspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\OD_Matrix.gdb"
    targetFeatures = os.path.join(inWorkspace1, "Hos2Naika_Chugoku")
    joinFeatures = os.path.join(inWorkspace2, "Census2015PopDenChugoku")
    
    # Output will be the target features: PHCCover Polygon with a sum of population field 
    outfc = os.path.join(outWorkdspace, "Hos2_NaikaChugoku_PopDen")
    
    # Create a new fieldmappings and add the two input feature classes.
    fieldmappings = arcpy.FieldMappings()
    fieldmappings.addTable(targetFeatures)
    fieldmappings.addTable(joinFeatures)
       
    # Delete fields that are no longer applicable, such as city CITY_NAME and CITY_FIPS
    # as only the first value will be used by default
    x = fieldmappings.findFieldMapIndex("P04_001")
    fieldmappings.removeFieldMap(x)
    y = fieldmappings.findFieldMapIndex("P04_002")
    fieldmappings.removeFieldMap(y)
    z = fieldmappings.findFieldMapIndex("P04_003")
    fieldmappings.removeFieldMap(z)
    c = fieldmappings.findFieldMapIndex("P04_004")
    fieldmappings.removeFieldMap(c)
    d = fieldmappings.findFieldMapIndex("P04_005")
    fieldmappings.removeFieldMap(d)
    e = fieldmappings.findFieldMapIndex("P04_006")
    fieldmappings.removeFieldMap(e)
    f = fieldmappings.findFieldMapIndex("P04_007")
    fieldmappings.removeFieldMap(f)
    h = fieldmappings.findFieldMapIndex("Naika")
    fieldmappings.removeFieldMap(h)

    #Run the Spatial Join tool, using the defaults for the join operation and join type
    arcpy.SpatialJoin_analysis(targetFeatures, joinFeatures, outfc, "JOIN_ONE_TO_ONE", "KEEP_ALL", fieldmappings,"COMPLETELY_WITHIN")

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [25]:
### Import system modules
import arcpy
import os

try:
    
    # Set local variables
    inWorkspace1 = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\HealthPlaceNew.gdb"
    inWorkspace2 = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\OD_Matrix.gdb"
    outWorkdspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\OD_Matrix.gdb"
    targetFeatures = os.path.join(inWorkspace1, "Hos3Naika_Chugoku")
    joinFeatures = os.path.join(inWorkspace2, "Census2015PopDenChugoku")
    
    # Output will be the target features: PHCCover Polygon with a sum of population field 
    outfc = os.path.join(outWorkdspace, "Hos3_NaikaChugoku_PopDen")
    
    # Create a new fieldmappings and add the two input feature classes.
    fieldmappings = arcpy.FieldMappings()
    fieldmappings.addTable(targetFeatures)
    fieldmappings.addTable(joinFeatures)
       
    # Delete fields that are no longer applicable, such as city CITY_NAME and CITY_FIPS
    # as only the first value will be used by default
    x = fieldmappings.findFieldMapIndex("P04_001")
    fieldmappings.removeFieldMap(x)
    y = fieldmappings.findFieldMapIndex("P04_002")
    fieldmappings.removeFieldMap(y)
    z = fieldmappings.findFieldMapIndex("P04_003")
    fieldmappings.removeFieldMap(z)
    c = fieldmappings.findFieldMapIndex("P04_004")
    fieldmappings.removeFieldMap(c)
    d = fieldmappings.findFieldMapIndex("P04_005")
    fieldmappings.removeFieldMap(d)
    e = fieldmappings.findFieldMapIndex("P04_006")
    fieldmappings.removeFieldMap(e)
    f = fieldmappings.findFieldMapIndex("P04_007")
    fieldmappings.removeFieldMap(f)
    h = fieldmappings.findFieldMapIndex("Naika")
    fieldmappings.removeFieldMap(h)

    #Run the Spatial Join tool, using the defaults for the join operation and join type
    arcpy.SpatialJoin_analysis(targetFeatures, joinFeatures, outfc, "JOIN_ONE_TO_ONE", "KEEP_ALL", fieldmappings,"COMPLETELY_WITHIN")

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


### Create OD Cost matrix layer

In [ ]:
# PHC

In [55]:
#Import system modules
import arcpy
from arcpy import env
import os

try:
       
    #Set environment settings
    output_dir = "C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew"
    #The NA layer's data will be saved to the workspace specified here
    env.workspace = os.path.join(output_dir, "OD_Matrix.gdb")
    env.overwriteOutput = True

    #Set inputs and outputs
    input_gdb1 = r"C:\Users\kenta\Documents\GISData_Import\Esri道路2014中国\道路網_鳥取県_島根県_岡山県_広島県_山口県版\ADF2014.gdb"
    input_gdb2 = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/HealthPlaceNew.gdb"
    input_gdb3 = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/OD_Matrix.gdb"
    network = os.path.join(input_gdb1, "ADF", "ADF_ND")
    origins = os.path.join(input_gdb3, "PHC_NaikaChugoku_PopDen")
    destinations = os.path.join(input_gdb3, "Census2015Res_CentroidChugoku")
    output_features = "PHCCoverpop"

    #Define some OD cost matrix analysis settings
    layer_name = "PHCCover"
    #User settings for driving
    travel_mode = "Driving Time"
    cut_off = "60"
     # number_destinations
     # time_of_day
     # time_zone
    line_shape = "NO_LINES"
     # accumulate_attribute
        
    #Create a new OD cost matrix layer.
    result_object = arcpy.na.MakeODCostMatrixAnalysisLayer(network, layer_name, travel_mode, cut_off,"","","",line_shape)

    #Get the layer object from the result object. 
    layer_object = result_object.getOutput(0)

    #Get the names of all the sublayers within the OD layer.
    sublayer_names = arcpy.na.GetNAClassNames(layer_object)
    #Store the layer names for later use
    origins_layer_name = sublayer_names["Origins"]
    destinations_layer_name = sublayer_names["Destinations"]

    #Add the address fields to the "origin" sublayers
    arcpy.na.AddFieldToAnalysisLayer(layer_object, origins_layer_name, "PHCID", "TEXT")
    arcpy.na.AddFieldToAnalysisLayer(layer_object, origins_layer_name,"PopDen", "DOUBLE")
    
    field_mappings = arcpy.na.NAClassFieldMappings(layer_object, origins_layer_name)
    field_mappings["PHCID"].mappedFieldName = "OBJECTID"
    field_mappings["PopDen"].mappedFieldName = "Census2015Res_CentroidChugoku_PopDen"

    #Load the census tracts as origins.
    arcpy.na.AddLocations(layer_object, origins_layer_name, origins, field_mappings)

    #Map the input number of population for each census centroid point
    #Destinations
    arcpy.na.AddFieldToAnalysisLayer(layer_object, destinations_layer_name,"Pop", "DOUBLE")
    
    field_mappings = arcpy.na.NAClassFieldMappings(layer_object,destinations_layer_name)
    field_mappings["Pop"].mappedFieldName = "TOTPOP_H27"
  
    #Load the hospitals as desinations.
    arcpy.na.AddLocations(layer_object, destinations_layer_name, destinations,field_mappings)

    #Solve the OD layer
    arcpy.na.Solve(layer_object)

    #Get sublayers
    #listLayers returns a list of sublayer layer objects contained in the NA
    #group layer, filtered by layer name used as a wildcard. Use the sublayer
    #name from GetNAClassNames as the wildcard string in case the sublayers
    #have non-default names.
    origins_sublayer = layer_object.listLayers(origins_layer_name)[0]
    destinations_sublayer = layer_object.listLayers(destinations_layer_name)[0]
    lines_sublayer = layer_object.listLayers(sublayer_names["ODLines"])[0]

    #Use the JoinField tool to transfer OD Cost Matrix information to the output feature class
    #Transfer the Hospital ID and PopDen fields Origins to the output Lines
    arcpy.management.JoinField(lines_sublayer, "OriginID",origins_sublayer, "ObjectID", "PHCID")
    arcpy.management.JoinField(lines_sublayer, "OriginID",origins_sublayer, "ObjectID", "PopDen")
    #Transfer the Pop field from the input Destinations to the output Lines
    arcpy.management.JoinField(lines_sublayer, "DestinationID",destinations_sublayer, "ObjectID", "Pop")
    
    print("Script completed successfully")

except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [ ]:
# Hos 2

In [36]:
#Import system modules
import arcpy
from arcpy import env
import os

try:
       
    #Set environment settings
    output_dir = "C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew"
    #The NA layer's data will be saved to the workspace specified here
    env.workspace = os.path.join(output_dir, "OD_Matrix.gdb")
    env.overwriteOutput = True

    #Set inputs and outputs
    input_gdb1 = r"C:\Users\kenta\Documents\GISData_Import\Esri道路2014中国\道路網_鳥取県_島根県_岡山県_広島県_山口県版\ADF2014.gdb"
    input_gdb2 = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/HealthPlaceNew.gdb"
    input_gdb3 = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/OD_Matrix.gdb"
    network = os.path.join(input_gdb1, "ADF", "ADF_ND")
    origins = os.path.join(input_gdb3, "Hos2_NaikaChugoku_PopDen")
    destinations = os.path.join(input_gdb3, "Census2015Res_CentroidChugoku")
    output_features = "Hos2Coverpop"

    #Define some OD cost matrix analysis settings
    layer_name = "Hos2Cover"
    #User settings for driving
    travel_mode = "Driving Time"
    cut_off = "60"
     # number_destinations
     # time_of_day
     # time_zone
    line_shape = "NO_LINES"
     # accumulate_attribute
        
    #Create a new OD cost matrix layer.
    result_object = arcpy.na.MakeODCostMatrixAnalysisLayer(network, layer_name, travel_mode, cut_off,"","","",line_shape)

    #Get the layer object from the result object. 
    layer_object = result_object.getOutput(0)

    #Get the names of all the sublayers within the OD layer.
    sublayer_names = arcpy.na.GetNAClassNames(layer_object)
    #Store the layer names for later use
    origins_layer_name = sublayer_names["Origins"]
    destinations_layer_name = sublayer_names["Destinations"]

    #Add the address fields to the "origin" sublayers
    arcpy.na.AddFieldToAnalysisLayer(layer_object, origins_layer_name, "Hos2ID", "TEXT")
    arcpy.na.AddFieldToAnalysisLayer(layer_object, origins_layer_name,"PopDen", "DOUBLE")
    
    field_mappings = arcpy.na.NAClassFieldMappings(layer_object, origins_layer_name)
    field_mappings["Hos2ID"].mappedFieldName = "OBJECTID"
    field_mappings["PopDen"].mappedFieldName = "Census2015Res_CentroidChugoku_PopDen"

    #Load the census tracts as origins.
    arcpy.na.AddLocations(layer_object, origins_layer_name, origins, field_mappings)

    #Map the input number of population for each census centroid point
    #Destinations
    arcpy.na.AddFieldToAnalysisLayer(layer_object, destinations_layer_name,"Pop", "DOUBLE")
    
    field_mappings = arcpy.na.NAClassFieldMappings(layer_object,destinations_layer_name)
    field_mappings["Pop"].mappedFieldName = "TOTPOP_H27"
  
    #Load the hospitals as desinations.
    arcpy.na.AddLocations(layer_object, destinations_layer_name, destinations,field_mappings)

    #Solve the OD layer
    arcpy.na.Solve(layer_object)

    #Get sublayers
    #listLayers returns a list of sublayer layer objects contained in the NA
    #group layer, filtered by layer name used as a wildcard. Use the sublayer
    #name from GetNAClassNames as the wildcard string in case the sublayers
    #have non-default names.
    origins_sublayer = layer_object.listLayers(origins_layer_name)[0]
    destinations_sublayer = layer_object.listLayers(destinations_layer_name)[0]
    lines_sublayer = layer_object.listLayers(sublayer_names["ODLines"])[0]

    #Use the JoinField tool to transfer OD Cost Matrix information to the output feature class
    #Transfer the Hospital ID and PopDen fields Origins to the output Lines
    arcpy.management.JoinField(lines_sublayer, "OriginID",origins_sublayer, "ObjectID", "Hos2ID")
    arcpy.management.JoinField(lines_sublayer, "OriginID",origins_sublayer, "ObjectID", "PopDen")
    #Transfer the Pop field from the input Destinations to the output Lines
    arcpy.management.JoinField(lines_sublayer, "DestinationID",destinations_sublayer, "ObjectID", "Pop")
    
    print("Script completed successfully")

except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [ ]:
# Hos 3

In [29]:
#Import system modules
import arcpy
from arcpy import env
import os

try:
       
    #Set environment settings
    output_dir = "C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew"
    #The NA layer's data will be saved to the workspace specified here
    env.workspace = os.path.join(output_dir, "OD_Matrix.gdb")
    env.overwriteOutput = True

    #Set inputs and outputs
    input_gdb1 = r"C:\Users\kenta\Documents\GISData_Import\Esri道路2014中国\道路網_鳥取県_島根県_岡山県_広島県_山口県版\ADF2014.gdb"
    input_gdb2 = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/HealthPlaceNew.gdb"
    input_gdb3 = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/OD_Matrix.gdb"
    network = os.path.join(input_gdb1, "ADF", "ADF_ND")
    origins = os.path.join(input_gdb3, "Hos3_NaikaChugoku_PopDen")
    destinations = os.path.join(input_gdb3, "Census2015Res_CentroidChugoku")
    output_features = "Hos3Coverpop"

    #Define some OD cost matrix analysis settings
    layer_name = "Hos3Cover"
    #User settings for driving
    travel_mode = "Driving Time"
    cut_off = "60"
     # number_destinations
     # time_of_day
     # time_zone
    line_shape = "NO_LINES"
     # accumulate_attribute
        
    #Create a new OD cost matrix layer.
    result_object = arcpy.na.MakeODCostMatrixAnalysisLayer(network, layer_name, travel_mode, cut_off,"","","",line_shape)

    #Get the layer object from the result object. 
    layer_object = result_object.getOutput(0)

    #Get the names of all the sublayers within the OD layer.
    sublayer_names = arcpy.na.GetNAClassNames(layer_object)
    #Store the layer names for later use
    origins_layer_name = sublayer_names["Origins"]
    destinations_layer_name = sublayer_names["Destinations"]

    #Add the address fields to the "origin" sublayers
    arcpy.na.AddFieldToAnalysisLayer(layer_object, origins_layer_name, "Hos3ID", "TEXT")
    arcpy.na.AddFieldToAnalysisLayer(layer_object, origins_layer_name,"PopDen", "DOUBLE")
    
    field_mappings = arcpy.na.NAClassFieldMappings(layer_object, origins_layer_name)
    field_mappings["Hos3ID"].mappedFieldName = "OBJECTID"
    field_mappings["PopDen"].mappedFieldName = "Census2015Res_CentroidChugoku_PopDen"

    #Load the census tracts as origins.
    arcpy.na.AddLocations(layer_object, origins_layer_name, origins, field_mappings)

    #Map the input number of population for each census centroid point
    #Destinations
    arcpy.na.AddFieldToAnalysisLayer(layer_object, destinations_layer_name,"Pop", "DOUBLE")
    
    field_mappings = arcpy.na.NAClassFieldMappings(layer_object,destinations_layer_name)
    field_mappings["Pop"].mappedFieldName = "TOTPOP_H27"
  
    #Load the hospitals as desinations.
    arcpy.na.AddLocations(layer_object, destinations_layer_name, destinations,field_mappings)

    #Solve the OD layer
    arcpy.na.Solve(layer_object)

    #Get sublayers
    #listLayers returns a list of sublayer layer objects contained in the NA
    #group layer, filtered by layer name used as a wildcard. Use the sublayer
    #name from GetNAClassNames as the wildcard string in case the sublayers
    #have non-default names.
    origins_sublayer = layer_object.listLayers(origins_layer_name)[0]
    destinations_sublayer = layer_object.listLayers(destinations_layer_name)[0]
    lines_sublayer = layer_object.listLayers(sublayer_names["ODLines"])[0]

    #Use the JoinField tool to transfer OD Cost Matrix information to the output feature class
    #Transfer the Hospital ID and PopDen fields Origins to the output Lines
    arcpy.management.JoinField(lines_sublayer, "OriginID",origins_sublayer, "ObjectID", "Hos3ID")
    arcpy.management.JoinField(lines_sublayer, "OriginID",origins_sublayer, "ObjectID", "PopDen")
    #Transfer the Pop field from the input Destinations to the output Lines
    arcpy.management.JoinField(lines_sublayer, "DestinationID",destinations_sublayer, "ObjectID", "Pop")
    
    print("Script completed successfully")

except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


### Weight the number of stores within 30 minutes based on Continuous distance-decay weight

In [ ]:
# PHC

In [1]:
import arcpy
from arcpy import env
try:
    # Set environment settings
    env.workspace =  r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/OD_Matrix.gdb"
    
    # Set local variables
    inTable = "ODLines17"
    fieldName1 = "Pop_wSlow"
    fieldName2 = "Pop_wQuick"
    expression = "w(!Total_所要時間（自動車）!,!PopDen!,!Pop!)"
    codeblock1 = """def w(f1,f2,f3):
        if f1 < 10:
            return f3
        elif (f2 >= 1500) and (f1 >= 10 and f1 < 30):
            return f3 * ((30-f1)/(30-10))**1.5
        elif (f2 >= 1500) and (f1 >= 30):
            return 0
        elif (f2 >= 1000 and f2 < 1500) and (f1 >= 10 and f1 < 45):
            return f3 * ((45-f1)/(45-10))**1.5
        elif (f2 >= 1000 and f2 < 1500) and (f1 >= 45):
            return 0
        elif (f2 < 1000) and (f1 >= 10 and f1 < 60):
            return f3 * ((60-f1)/(60-10))**1.5
        elif (f2 < 1000) and (f1 >= 60):
            return 0"""
    codeblock2 = """def w(f1,f2,f3):
        if f1 < 10:
            return f3
        elif (f2 >= 1500) and (f1 >= 10 and f1 < 30):
            return f3 * ((30-f1)/(30-10))**2
        elif (f2 >= 1500) and (f1 >= 30):
            return 0
        elif (f2 >= 1000 and f2 < 1500) and (f1 >= 10 and f1 < 45):
            return f3 * ((45-f1)/(45-10))**2
        elif (f2 >= 1000 and f2 < 1500) and (f1 >= 45):
            return 0
        elif (f2 < 1000) and (f1 >= 10 and f1 < 60):
            return f3 * ((60-f1)/(60-10))**2
        elif (f2 < 1000) and (f1 >= 60):
            return 0"""

    # Execute AddField
    arcpy.AddField_management(inTable, fieldName1, "DOUBLE")
    arcpy.AddField_management(inTable, fieldName2, "DOUBLE")
    # Execute CalculateField - weighting
    arcpy.CalculateField_management(inTable, fieldName1, expression, "PYTHON_9.3", codeblock1)
    arcpy.CalculateField_management(inTable, fieldName2, expression, "PYTHON_9.3", codeblock2)

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [ ]:
# Hos 2

In [37]:
import arcpy
from arcpy import env
try:
    # Set environment settings
    env.workspace =  r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/OD_Matrix.gdb"
    
    # Set local variables
    inTable = "ODLines16"
    fieldName1 = "Pop_wSlow"
    fieldName2 = "Pop_wQuick"
    expression = "w(!Total_所要時間（自動車）!,!PopDen!,!Pop!)"
    codeblock1 = """def w(f1,f2,f3):
        if f1 < 10:
            return f3
        elif (f2 >= 1500) and (f1 >= 10 and f1 < 30):
            return f3 * ((30-f1)/(30-10))**1.5
        elif (f2 >= 1500) and (f1 >= 30):
            return 0
        elif (f2 >= 1000 and f2 < 1500) and (f1 >= 10 and f1 < 45):
            return f3 * ((45-f1)/(45-10))**1.5
        elif (f2 >= 1000 and f2 < 1500) and (f1 >= 45):
            return 0
        elif (f2 < 1000) and (f1 >= 10 and f1 < 60):
            return f3 * ((60-f1)/(60-10))**1.5
        elif (f2 < 1000) and (f1 >= 60):
            return 0"""
    codeblock2 = """def w(f1,f2,f3):
        if f1 < 10:
            return f3
        elif (f2 >= 1500) and (f1 >= 10 and f1 < 30):
            return f3 * ((30-f1)/(30-10))**2
        elif (f2 >= 1500) and (f1 >= 30):
            return 0
        elif (f2 >= 1000 and f2 < 1500) and (f1 >= 10 and f1 < 45):
            return f3 * ((45-f1)/(45-10))**2
        elif (f2 >= 1000 and f2 < 1500) and (f1 >= 45):
            return 0
        elif (f2 < 1000) and (f1 >= 10 and f1 < 60):
            return f3 * ((60-f1)/(60-10))**2
        elif (f2 < 1000) and (f1 >= 60):
            return 0"""

    # Execute AddField
    arcpy.AddField_management(inTable, fieldName1, "DOUBLE")
    arcpy.AddField_management(inTable, fieldName2, "DOUBLE")
    # Execute CalculateField - weighting
    arcpy.CalculateField_management(inTable, fieldName1, expression, "PYTHON_9.3", codeblock1)
    arcpy.CalculateField_management(inTable, fieldName2, expression, "PYTHON_9.3", codeblock2)

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [ ]:
# Hos 3

In [30]:
import arcpy
from arcpy import env
try:
    # Set environment settings
    env.workspace =  r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/OD_Matrix.gdb"
    
    # Set local variables
    inTable = "ODLines15"
    fieldName1 = "Pop_wSlow"
    fieldName2 = "Pop_wQuick"
    expression = "w(!Total_所要時間（自動車）!,!PopDen!,!Pop!)"
    codeblock1 = """def w(f1,f2,f3):
        if f1 < 10:
            return f3
        elif (f2 >= 1500) and (f1 >= 10 and f1 < 30):
            return f3 * ((30-f1)/(30-10))**1.5
        elif (f2 >= 1500) and (f1 >= 30):
            return 0
        elif (f2 >= 1000 and f2 < 1500) and (f1 >= 10 and f1 < 45):
            return f3 * ((45-f1)/(45-10))**1.5
        elif (f2 >= 1000 and f2 < 1500) and (f1 >= 45):
            return 0
        elif (f2 < 1000) and (f1 >= 10 and f1 < 60):
            return f3 * ((60-f1)/(60-10))**1.5
        elif (f2 < 1000) and (f1 >= 60):
            return 0"""
    codeblock2 = """def w(f1,f2,f3):
        if f1 < 10:
            return f3
        elif (f2 >= 1500) and (f1 >= 10 and f1 < 30):
            return f3 * ((30-f1)/(30-10))**2
        elif (f2 >= 1500) and (f1 >= 30):
            return 0
        elif (f2 >= 1000 and f2 < 1500) and (f1 >= 10 and f1 < 45):
            return f3 * ((45-f1)/(45-10))**2
        elif (f2 >= 1000 and f2 < 1500) and (f1 >= 45):
            return 0
        elif (f2 < 1000) and (f1 >= 10 and f1 < 60):
            return f3 * ((60-f1)/(60-10))**2
        elif (f2 < 1000) and (f1 >= 60):
            return 0"""

    # Execute AddField
    arcpy.AddField_management(inTable, fieldName1, "DOUBLE")
    arcpy.AddField_management(inTable, fieldName2, "DOUBLE")
    # Execute CalculateField - weighting
    arcpy.CalculateField_management(inTable, fieldName1, expression, "PYTHON_9.3", codeblock1)
    arcpy.CalculateField_management(inTable, fieldName2, expression, "PYTHON_9.3", codeblock2)

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


### Summary statistics by Origin ID to sum up the weighted population

In [2]:
# Import system modules
import arcpy
from arcpy import env
 
import os
try:
    # Set environment settings
    env.workspace = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/OD_Matrix.gdb"
    # Set local variables
    inFeatures = "ODLines17"
    outStatsTable = "PHCCover"
    statsFields = [["Pop_wSlow","sum"],["Pop_wQuick","sum"]]
    case_field = "OriginID"
    # Execute Statistics to get the area of each vegetation type within the clipped buffer.
    arcpy.Statistics_analysis(inFeatures, outStatsTable, statsFields, case_field)

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [ ]:
# Hos2

In [52]:
# Import system modules
import arcpy
from arcpy import env
 
import os
try:
    # Set environment settings
    env.workspace = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/OD_Matrix.gdb"
    # Set local variables
    inFeatures = "ODLines16"
    outStatsTable = "Hos2Cover"
    statsFields = [["Pop_wSlow","sum"],["Pop_wQuick","sum"]]
    case_field = "OriginID"
    # Execute Statistics to get the area of each vegetation type within the clipped buffer.
    arcpy.Statistics_analysis(inFeatures, outStatsTable, statsFields, case_field)

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [ ]:
# Hos3

In [48]:
# Import system modules
import arcpy
from arcpy import env
 
import os
try:
    # Set environment settings
    env.workspace = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/OD_Matrix.gdb"
    # Set local variables
    inFeatures = "ODLines15"
    outStatsTable = "Hos3Cover"
    statsFields = [["Pop_wSlow","sum"],["Pop_wQuick","sum"]]
    case_field = "OriginID"
    # Execute Statistics to get the area of each vegetation type within the clipped buffer.
    arcpy.Statistics_analysis(inFeatures, outStatsTable, statsFields, case_field)

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [ ]:
# Get Provider to Population ratio

### Join the PropToPop back to PHCPoint

In [1]:
import arcpy
from arcpy import env
import os
try:
    # Set environment settings
    env.workspace = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/OD_Matrix.gdb"
    
    # Set the local parameters
    inWorkspace1 = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/OD_Matrix.gdb"
    inWorkspace2 = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/OD_Matrix.gdb"
    inFeatures = os.path.join(inWorkspace1,"PHC_NaikaChugoku_PopDen")
    joinField1 = "OBJECTID"
    joinTable = os.path.join(inWorkspace2,"PHCCover")
    joinField2 = "OriginID"
    fieldList = ["SUM_Pop_wSlow","SUM_Pop_wQuick"] 
    outFeatures = os.path.join(inWorkspace2,"PHCCover_PTP")
    #dropFields1 = ["OriginID"]
    # Join two feature classes by the OBJECTID and OriginID fields and only carry 
    # over the SUM_SuperCount field
    arcpy.JoinField_management(inFeatures, joinField1, joinTable, joinField2, fieldList)
    # Create a new feature in a current workspace = E2FCAStep1Rural.gdb
    arcpy.CopyFeatures_management(inFeatures, outFeatures)
    # Delete unnecessary fields
    #arcpy.DeleteField_management(outFeatures, dropFields1)
    
    print("Script completed successfully")

except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [2]:
import arcpy
from arcpy import env
try:
    # Set environment settings
    env.workspace = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/OD_Matrix.gdb"
    
    # Set local variables
    inTable = "PHCCover_PTP"
    fieldName1 = "SUM_Pop_wSlow"
    fieldName2 = "SUM_Pop_wQuick"
    expression1 = "1/!SUM_Pop_wSlow!"
    expression2 = "1/!SUM_Pop_wQuick!"    
    # Execute CalculateField - getting PropToPop deviding 1 by weighted population
    arcpy.CalculateField_management(inTable, fieldName1, expression1, "PYTHON_9.3")
    arcpy.CalculateField_management(inTable, fieldName2, expression2, "PYTHON_9.3")

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [ ]:
# Hos2

In [53]:
import arcpy
from arcpy import env
import os
try:
    # Set environment settings
    env.workspace = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/OD_Matrix.gdb"
    
    # Set the local parameters
    inWorkspace1 = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/OD_Matrix.gdb"
    inWorkspace2 = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/OD_Matrix.gdb"
    inFeatures = os.path.join(inWorkspace1,"Hos2_NaikaChugoku_PopDen")
    joinField1 = "OBJECTID"
    joinTable = os.path.join(inWorkspace2,"Hos2Cover")
    joinField2 = "OriginID"
    fieldList = ["SUM_Pop_wSlow","SUM_Pop_wQuick"] 
    outFeatures = os.path.join(inWorkspace2,"Hos2Cover_PTP")
    #dropFields1 = ["OriginID"]
    # Join two feature classes by the OBJECTID and OriginID fields and only carry 
    # over the SUM_SuperCount field
    arcpy.JoinField_management(inFeatures, joinField1, joinTable, joinField2, fieldList)
    # Create a new feature in a current workspace = E2FCAStep1Rural.gdb
    arcpy.CopyFeatures_management(inFeatures, outFeatures)
    # Delete unnecessary fields
    #arcpy.DeleteField_management(outFeatures, dropFields1)
    
    print("Script completed successfully")

except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [54]:
import arcpy
from arcpy import env
try:
    # Set environment settings
    env.workspace = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/OD_Matrix.gdb"
    
    # Set local variables
    inTable = "Hos2Cover_PTP"
    fieldName1 = "SUM_Pop_wSlow"
    fieldName2 = "SUM_Pop_wQuick"
    expression1 = "1/!SUM_Pop_wSlow!"
    expression2 = "1/!SUM_Pop_wQuick!"    
    # Execute CalculateField - getting PropToPop deviding 1 by weighted population
    arcpy.CalculateField_management(inTable, fieldName1, expression1, "PYTHON_9.3")
    arcpy.CalculateField_management(inTable, fieldName2, expression2, "PYTHON_9.3")

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [ ]:
# Hos3

In [50]:
import arcpy
from arcpy import env
import os
try:
    # Set environment settings
    env.workspace = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/OD_Matrix.gdb"
    
    # Set the local parameters
    inWorkspace1 = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/OD_Matrix.gdb"
    inWorkspace2 = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/OD_Matrix.gdb"
    inFeatures = os.path.join(inWorkspace1,"Hos3_NaikaChugoku_PopDen")
    joinField1 = "OBJECTID"
    joinTable = os.path.join(inWorkspace2,"Hos3Cover")
    joinField2 = "OriginID"
    fieldList = ["SUM_Pop_wSlow","SUM_Pop_wQuick"] 
    outFeatures = os.path.join(inWorkspace2,"Hos3Cover_PTP")
    #dropFields1 = ["OriginID"]
    # Join two feature classes by the OBJECTID and OriginID fields and only carry 
    # over the SUM_SuperCount field
    arcpy.JoinField_management(inFeatures, joinField1, joinTable, joinField2, fieldList)
    # Create a new feature in a current workspace = E2FCAStep1Rural.gdb
    arcpy.CopyFeatures_management(inFeatures, outFeatures)
    # Delete unnecessary fields
    #arcpy.DeleteField_management(outFeatures, dropFields1)
    
    print("Script completed successfully")

except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [51]:
import arcpy
from arcpy import env
try:
    # Set environment settings
    env.workspace = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/OD_Matrix.gdb"
    
    # Set local variables
    inTable = "Hos3Cover_PTP"
    fieldName1 = "SUM_Pop_wSlow"
    fieldName2 = "SUM_Pop_wQuick"
    expression1 = "1/!SUM_Pop_wSlow!"
    expression2 = "1/!SUM_Pop_wQuick!"    
    # Execute CalculateField - getting PropToPop deviding 1 by weighted population
    arcpy.CalculateField_management(inTable, fieldName1, expression1, "PYTHON_9.3")
    arcpy.CalculateField_management(inTable, fieldName2, expression2, "PYTHON_9.3")

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


# Create OD Cost Matrix from Census centroid point to PHC

In [ ]:
# PHC

In [1]:
#Import system modules
import arcpy
from arcpy import env
import os

try:
       
    #Set environment settings
    output_dir = "C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew"
    #The NA layer's data will be saved to the workspace specified here
    env.workspace = os.path.join(output_dir, "OD_Matrix.gdb")
    env.overwriteOutput = True

    #Set inputs and outputs
    input_gdb1 = r"C:\Users\kenta\Documents\GISData_Import\Esri道路2014中国\道路網_鳥取県_島根県_岡山県_広島県_山口県版\ADF2014.gdb"
    input_gdb2 = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/HealthPlaceNew.gdb"
    input_gdb3 = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/OD_Matrix.gdb"
    network = os.path.join(input_gdb1, "ADF", "ADF_ND")
    origins = os.path.join(input_gdb3, "Census2015Res_CentroidChugoku")
    destinations = os.path.join(input_gdb3, "PHCCover_PTP")
    output_features = "CentroidCoverPHC"

    #Define some OD cost matrix analysis settings
    layer_name = "CentroidCoverPHC"
    #User settings for driving
    travel_mode = "Driving Time"
    cut_off = "60"
     # number_destinations
     # time_of_day
     # time_zone
    line_shape = "NO_LINES"
     # accumulate_attribute
        
    #Create a new OD cost matrix layer.
    result_object = arcpy.na.MakeODCostMatrixAnalysisLayer(network, layer_name, travel_mode, cut_off,"","","",line_shape)

    #Get the layer object from the result object. 
    layer_object = result_object.getOutput(0)

    #Get the names of all the sublayers within the OD layer.
    sublayer_names = arcpy.na.GetNAClassNames(layer_object)
    #Store the layer names for later use
    origins_layer_name = sublayer_names["Origins"]
    destinations_layer_name = sublayer_names["Destinations"]

    #Add the address fields to the "origin" sublayers
    arcpy.na.AddFieldToAnalysisLayer(layer_object, origins_layer_name, "CensusID","TEXT")
    arcpy.na.AddFieldToAnalysisLayer(layer_object, origins_layer_name,"PopDen", "DOUBLE")
    
    field_mappings = arcpy.na.NAClassFieldMappings(layer_object, origins_layer_name)
    field_mappings["CensusID"].mappedFieldName = "OBJECTID"
    field_mappings["PopDen"].mappedFieldName = "PopDen"

    #Load the census tracts as origins.
    arcpy.na.AddLocations(layer_object, origins_layer_name, origins, field_mappings)

    #Map the input number of population for each census centroid point
    #Destinations
    arcpy.na.AddFieldToAnalysisLayer(layer_object, destinations_layer_name,"PropToPopSlow", "DOUBLE")
    arcpy.na.AddFieldToAnalysisLayer(layer_object, destinations_layer_name,"PropToPopQuick", "DOUBLE")
    
    field_mappings = arcpy.na.NAClassFieldMappings(layer_object,destinations_layer_name)
    field_mappings["PropToPopSlow"].mappedFieldName = "SUM_Pop_wSlow"
    field_mappings["PropToPopQuick"].mappedFieldName = "SUM_Pop_wQuick"

    #Load the hospitals as desinations.
    arcpy.na.AddLocations(layer_object, destinations_layer_name, destinations,field_mappings)

    #Solve the OD layer
    arcpy.na.Solve(layer_object)

    #Get sublayers
    #listLayers returns a list of sublayer layer objects contained in the NA
    #group layer, filtered by layer name used as a wildcard. Use the sublayer
    #name from GetNAClassNames as the wildcard string in case the sublayers
    #have non-default names.
    origins_sublayer = layer_object.listLayers(origins_layer_name)[0]
    destinations_sublayer = layer_object.listLayers(destinations_layer_name)[0]
    lines_sublayer = layer_object.listLayers(sublayer_names["ODLines"])[0]

    #Use the JoinField tool to transfer OD Cost Matrix information to the
    #output feature class
    #Transfer the tract ID from the input Origins to the output Lines
    arcpy.management.JoinField(lines_sublayer, "OriginID",origins_sublayer, "ObjectID", "CensusID")
    arcpy.management.JoinField(lines_sublayer, "OriginID",origins_sublayer, "ObjectID", "PopDen")
    #Transfer the hospital name from the input Destinations to the output Lines
    arcpy.management.JoinField(lines_sublayer, "DestinationID",destinations_sublayer, "ObjectID", "PropToPopSlow")
    arcpy.management.JoinField(lines_sublayer, "DestinationID",destinations_sublayer, "ObjectID", "PropToPopQuick")
    #Transfer fields of interest (hospital name, impedance attribute, and other
    #accumulated costs) from the output Lines to a copy of the input census
    #tracts feature class using the Tract_ID field
    # Determine the impedance attribute
 
    print("Script completed successfully")

except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [ ]:
# Hos2

In [4]:
#Import system modules
import arcpy
from arcpy import env
import os

try:
       
    #Set environment settings
    output_dir = "C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew"
    #The NA layer's data will be saved to the workspace specified here
    env.workspace = os.path.join(output_dir, "OD_Matrix.gdb")
    env.overwriteOutput = True

    #Set inputs and outputs
    input_gdb1 = r"C:\Users\kenta\Documents\GISData_Import\Esri道路2014中国\道路網_鳥取県_島根県_岡山県_広島県_山口県版\ADF2014.gdb"
    input_gdb2 = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/HealthPlaceNew.gdb"
    input_gdb3 = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/OD_Matrix.gdb"
    network = os.path.join(input_gdb1, "ADF", "ADF_ND")
    origins = os.path.join(input_gdb3, "Census2015Res_CentroidChugoku")
    destinations = os.path.join(input_gdb3, "Hos2Cover_PTP")
    output_features = "CentroidCoverHos2"

    #Define some OD cost matrix analysis settings
    layer_name = "CentroidCoverHos2"
    #User settings for driving
    travel_mode = "Driving Time"
    cut_off = "60"
     # number_destinations
     # time_of_day
     # time_zone
    line_shape = "NO_LINES"
     # accumulate_attribute
        
    #Create a new OD cost matrix layer.
    result_object = arcpy.na.MakeODCostMatrixAnalysisLayer(network, layer_name, travel_mode, cut_off,"","","",line_shape)

    #Get the layer object from the result object. 
    layer_object = result_object.getOutput(0)

    #Get the names of all the sublayers within the OD layer.
    sublayer_names = arcpy.na.GetNAClassNames(layer_object)
    #Store the layer names for later use
    origins_layer_name = sublayer_names["Origins"]
    destinations_layer_name = sublayer_names["Destinations"]

    #Add the address fields to the "origin" sublayers
    arcpy.na.AddFieldToAnalysisLayer(layer_object, origins_layer_name, "CensusID","TEXT")
    arcpy.na.AddFieldToAnalysisLayer(layer_object, origins_layer_name,"PopDen", "DOUBLE")
    
    field_mappings = arcpy.na.NAClassFieldMappings(layer_object, origins_layer_name)
    field_mappings["CensusID"].mappedFieldName = "OBJECTID"
    field_mappings["PopDen"].mappedFieldName = "PopDen"

    #Load the census tracts as origins.
    arcpy.na.AddLocations(layer_object, origins_layer_name, origins, field_mappings)

    #Map the input number of population for each census centroid point
    #Destinations
    arcpy.na.AddFieldToAnalysisLayer(layer_object, destinations_layer_name,"PropToPopSlow", "DOUBLE")
    arcpy.na.AddFieldToAnalysisLayer(layer_object, destinations_layer_name,"PropToPopQuick", "DOUBLE")
    
    field_mappings = arcpy.na.NAClassFieldMappings(layer_object,destinations_layer_name)
    field_mappings["PropToPopSlow"].mappedFieldName = "SUM_Pop_wSlow"
    field_mappings["PropToPopQuick"].mappedFieldName = "SUM_Pop_wQuick"

    #Load the hospitals as desinations.
    arcpy.na.AddLocations(layer_object, destinations_layer_name, destinations,field_mappings)

    #Solve the OD layer
    arcpy.na.Solve(layer_object)

    #Get sublayers
    #listLayers returns a list of sublayer layer objects contained in the NA
    #group layer, filtered by layer name used as a wildcard. Use the sublayer
    #name from GetNAClassNames as the wildcard string in case the sublayers
    #have non-default names.
    origins_sublayer = layer_object.listLayers(origins_layer_name)[0]
    destinations_sublayer = layer_object.listLayers(destinations_layer_name)[0]
    lines_sublayer = layer_object.listLayers(sublayer_names["ODLines"])[0]

    #Use the JoinField tool to transfer OD Cost Matrix information to the
    #output feature class
    #Transfer the tract ID from the input Origins to the output Lines
    arcpy.management.JoinField(lines_sublayer, "OriginID",origins_sublayer, "ObjectID", "CensusID")
    arcpy.management.JoinField(lines_sublayer, "OriginID",origins_sublayer, "ObjectID", "PopDen")
    #Transfer the hospital name from the input Destinations to the output Lines
    arcpy.management.JoinField(lines_sublayer, "DestinationID",destinations_sublayer, "ObjectID", "PropToPopSlow")
    arcpy.management.JoinField(lines_sublayer, "DestinationID",destinations_sublayer, "ObjectID", "PropToPopQuick")
    #Transfer fields of interest (hospital name, impedance attribute, and other
    #accumulated costs) from the output Lines to a copy of the input census
    #tracts feature class using the Tract_ID field
    # Determine the impedance attribute
 
    print("Script completed successfully")

except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [ ]:
# Hos3

In [3]:
#Import system modules
import arcpy
from arcpy import env
import os

try:
       
    #Set environment settings
    output_dir = "C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew"
    #The NA layer's data will be saved to the workspace specified here
    env.workspace = os.path.join(output_dir, "OD_Matrix.gdb")
    env.overwriteOutput = True

    #Set inputs and outputs
    input_gdb1 = r"C:\Users\kenta\Documents\GISData_Import\Esri道路2014中国\道路網_鳥取県_島根県_岡山県_広島県_山口県版\ADF2014.gdb"
    input_gdb2 = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/HealthPlaceNew.gdb"
    input_gdb3 = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/OD_Matrix.gdb"
    network = os.path.join(input_gdb1, "ADF", "ADF_ND")
    origins = os.path.join(input_gdb3, "Census2015Res_CentroidChugoku")
    destinations = os.path.join(input_gdb3, "Hos3Cover_PTP")
    output_features = "CentroidCoverHos3"

    #Define some OD cost matrix analysis settings
    layer_name = "CentroidCoverHos3"
    #User settings for driving
    travel_mode = "Driving Time"
    cut_off = "60"
     # number_destinations
     # time_of_day
     # time_zone
    line_shape = "NO_LINES"
     # accumulate_attribute
        
    #Create a new OD cost matrix layer.
    result_object = arcpy.na.MakeODCostMatrixAnalysisLayer(network, layer_name, travel_mode, cut_off,"","","",line_shape)

    #Get the layer object from the result object. 
    layer_object = result_object.getOutput(0)

    #Get the names of all the sublayers within the OD layer.
    sublayer_names = arcpy.na.GetNAClassNames(layer_object)
    #Store the layer names for later use
    origins_layer_name = sublayer_names["Origins"]
    destinations_layer_name = sublayer_names["Destinations"]

    #Add the address fields to the "origin" sublayers
    arcpy.na.AddFieldToAnalysisLayer(layer_object, origins_layer_name, "CensusID","TEXT")
    arcpy.na.AddFieldToAnalysisLayer(layer_object, origins_layer_name,"PopDen", "DOUBLE")
    
    field_mappings = arcpy.na.NAClassFieldMappings(layer_object, origins_layer_name)
    field_mappings["CensusID"].mappedFieldName = "OBJECTID"
    field_mappings["PopDen"].mappedFieldName = "PopDen"

    #Load the census tracts as origins.
    arcpy.na.AddLocations(layer_object, origins_layer_name, origins, field_mappings)

    #Map the input number of population for each census centroid point
    #Destinations
    arcpy.na.AddFieldToAnalysisLayer(layer_object, destinations_layer_name,"PropToPopSlow", "DOUBLE")
    arcpy.na.AddFieldToAnalysisLayer(layer_object, destinations_layer_name,"PropToPopQuick", "DOUBLE")
    
    field_mappings = arcpy.na.NAClassFieldMappings(layer_object,destinations_layer_name)
    field_mappings["PropToPopSlow"].mappedFieldName = "SUM_Pop_wSlow"
    field_mappings["PropToPopQuick"].mappedFieldName = "SUM_Pop_wQuick"

    #Load the hospitals as desinations.
    arcpy.na.AddLocations(layer_object, destinations_layer_name, destinations,field_mappings)

    #Solve the OD layer
    arcpy.na.Solve(layer_object)

    #Get sublayers
    #listLayers returns a list of sublayer layer objects contained in the NA
    #group layer, filtered by layer name used as a wildcard. Use the sublayer
    #name from GetNAClassNames as the wildcard string in case the sublayers
    #have non-default names.
    origins_sublayer = layer_object.listLayers(origins_layer_name)[0]
    destinations_sublayer = layer_object.listLayers(destinations_layer_name)[0]
    lines_sublayer = layer_object.listLayers(sublayer_names["ODLines"])[0]

    #Use the JoinField tool to transfer OD Cost Matrix information to the
    #output feature class
    #Transfer the tract ID from the input Origins to the output Lines
    arcpy.management.JoinField(lines_sublayer, "OriginID",origins_sublayer, "ObjectID", "CensusID")
    arcpy.management.JoinField(lines_sublayer, "OriginID",origins_sublayer, "ObjectID", "PopDen")
    #Transfer the hospital name from the input Destinations to the output Lines
    arcpy.management.JoinField(lines_sublayer, "DestinationID",destinations_sublayer, "ObjectID", "PropToPopSlow")
    arcpy.management.JoinField(lines_sublayer, "DestinationID",destinations_sublayer, "ObjectID", "PropToPopQuick")
    #Transfer fields of interest (hospital name, impedance attribute, and other
    #accumulated costs) from the output Lines to a copy of the input census
    #tracts feature class using the Tract_ID field
    # Determine the impedance attribute
 
    print("Script completed successfully")

except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


# Weight the score by continuous distance decay

In [ ]:
# PHC

In [1]:
import arcpy
from arcpy import env
try:
    # Set environment settings
    env.workspace =  r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/OD_Matrix.gdb"
    
    # Set local variables
    inTable = "ODLines3"
    fieldName1 = "E2FCA_OD_PHCSlow"
    fieldName2 = "E2FCA_OD_PHCQuick"
    expression = "w(!Total_所要時間（自動車）!,!PopDen!,!PropToPopSlow!)"
    expression = "w(!Total_所要時間（自動車）!,!PopDen!,!PropToPopQuick!)"
    codeblock1 = """def w(f1,f2,f3):
        if f1 < 10:
            return f3
        elif (f2 >= 1500) and (f1 >= 10 and f1 < 30):
            return f3 * ((30-f1)/(30-10))**1.5
        elif (f2 >= 1500) and (f1 >= 30):
            return 0
        elif (f2 >= 1000 and f2 < 1500) and (f1 >= 10 and f1 < 45):
            return f3 * ((45-f1)/(45-10))**1.5
        elif (f2 >= 1000 and f2 < 1500) and (f1 >= 45):
            return 0
        elif (f2 < 1000) and (f1 >= 10 and f1 < 60):
            return f3 * ((60-f1)/(60-10))**1.5
        elif (f2 < 1000) and (f1 >= 60):
            return 0"""
    codeblock2 = """def w(f1,f2,f3):
        if f1 < 10:
            return f3
        elif (f2 >= 1500) and (f1 >= 10 and f1 < 30):
            return f3 * ((30-f1)/(30-10))**2
        elif (f2 >= 1500) and (f1 >= 30):
            return 0
        elif (f2 >= 1000 and f2 < 1500) and (f1 >= 10 and f1 < 45):
            return f3 * ((45-f1)/(45-10))**2
        elif (f2 >= 1000 and f2 < 1500) and (f1 >= 45):
            return 0
        elif (f2 < 1000) and (f1 >= 10 and f1 < 60):
            return f3 * ((60-f1)/(60-10))**2
        elif (f2 < 1000) and (f1 >= 60):
            return 0"""

    # Execute AddField
    arcpy.AddField_management(inTable, fieldName1, "DOUBLE")
    arcpy.AddField_management(inTable, fieldName2, "DOUBLE")
    # Execute CalculateField - weighting
    arcpy.CalculateField_management(inTable, fieldName1, expression, "PYTHON_9.3", codeblock1)
    arcpy.CalculateField_management(inTable, fieldName2, expression, "PYTHON_9.3", codeblock2)

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [ ]:
# Hos2

In [2]:
import arcpy
from arcpy import env
try:
    # Set environment settings
    env.workspace =  r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/OD_Matrix.gdb"
    
    # Set local variables
    inTable = "ODLines2"
    fieldName1 = "E2FCA_OD_Hos2Slow"
    fieldName2 = "E2FCA_OD_Hos2Quick"
    expression1 = "w(!Total_所要時間（自動車）!,!PopDen!,!PropToPopSlow!)"
    expression2 = "w(!Total_所要時間（自動車）!,!PopDen!,!PropToPopQuick!)"
    codeblock1 = """def w(f1,f2,f3):
        if f1 < 10:
            return f3
        elif (f2 >= 1500) and (f1 >= 10 and f1 < 30):
            return f3 * ((30-f1)/(30-10))**1.5
        elif (f2 >= 1500) and (f1 >= 30):
            return 0
        elif (f2 >= 1000 and f2 < 1500) and (f1 >= 10 and f1 < 45):
            return f3 * ((45-f1)/(45-10))**1.5
        elif (f2 >= 1000 and f2 < 1500) and (f1 >= 45):
            return 0
        elif (f2 < 1000) and (f1 >= 10 and f1 < 60):
            return f3 * ((60-f1)/(60-10))**1.5
        elif (f2 < 1000) and (f1 >= 60):
            return 0"""
    codeblock2 = """def w(f1,f2,f3):
        if f1 < 10:
            return f3
        elif (f2 >= 1500) and (f1 >= 10 and f1 < 30):
            return f3 * ((30-f1)/(30-10))**2
        elif (f2 >= 1500) and (f1 >= 30):
            return 0
        elif (f2 >= 1000 and f2 < 1500) and (f1 >= 10 and f1 < 45):
            return f3 * ((45-f1)/(45-10))**2
        elif (f2 >= 1000 and f2 < 1500) and (f1 >= 45):
            return 0
        elif (f2 < 1000) and (f1 >= 10 and f1 < 60):
            return f3 * ((60-f1)/(60-10))**2
        elif (f2 < 1000) and (f1 >= 60):
            return 0"""

    # Execute AddField
    arcpy.AddField_management(inTable, fieldName1, "DOUBLE")
    arcpy.AddField_management(inTable, fieldName2, "DOUBLE")
    # Execute CalculateField - weighting
    arcpy.CalculateField_management(inTable, fieldName1, expression1, "PYTHON_9.3", codeblock1)
    arcpy.CalculateField_management(inTable, fieldName2, expression2, "PYTHON_9.3", codeblock2)

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [ ]:
# Hos3

In [3]:
import arcpy
from arcpy import env
try:
    # Set environment settings
    env.workspace =  r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/OD_Matrix.gdb"
    
    # Set local variables
    inTable = "ODLines1"
    fieldName1 = "E2FCA_OD_Hos3Slow"
    fieldName2 = "E2FCA_OD_Hos3Quick"
    expression1 = "w(!Total_所要時間（自動車）!,!PopDen!,!PropToPopSlow!)"
    expression2 = "w(!Total_所要時間（自動車）!,!PopDen!,!PropToPopQuick!)"
    codeblock1 = """def w(f1,f2,f3):
        if f1 < 10:
            return f3
        elif (f2 >= 1500) and (f1 >= 10 and f1 < 30):
            return f3 * ((30-f1)/(30-10))**1.5
        elif (f2 >= 1500) and (f1 >= 30):
            return 0
        elif (f2 >= 1000 and f2 < 1500) and (f1 >= 10 and f1 < 45):
            return f3 * ((45-f1)/(45-10))**1.5
        elif (f2 >= 1000 and f2 < 1500) and (f1 >= 45):
            return 0
        elif (f2 < 1000) and (f1 >= 10 and f1 < 60):
            return f3 * ((60-f1)/(60-10))**1.5
        elif (f2 < 1000) and (f1 >= 60):
            return 0"""
    codeblock2 = """def w(f1,f2,f3):
        if f1 < 10:
            return f3
        elif (f2 >= 1500) and (f1 >= 10 and f1 < 30):
            return f3 * ((30-f1)/(30-10))**2
        elif (f2 >= 1500) and (f1 >= 30):
            return 0
        elif (f2 >= 1000 and f2 < 1500) and (f1 >= 10 and f1 < 45):
            return f3 * ((45-f1)/(45-10))**2
        elif (f2 >= 1000 and f2 < 1500) and (f1 >= 45):
            return 0
        elif (f2 < 1000) and (f1 >= 10 and f1 < 60):
            return f3 * ((60-f1)/(60-10))**2
        elif (f2 < 1000) and (f1 >= 60):
            return 0"""

    # Execute AddField
    arcpy.AddField_management(inTable, fieldName1, "DOUBLE")
    arcpy.AddField_management(inTable, fieldName2, "DOUBLE")
    # Execute CalculateField - weighting
    arcpy.CalculateField_management(inTable, fieldName1, expression1, "PYTHON_9.3", codeblock1)
    arcpy.CalculateField_management(inTable, fieldName2, expression2, "PYTHON_9.3", codeblock2)

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


### Summary statistics by Origin ID to sum up the weighted PTP

In [ ]:
# PHC

In [4]:
# Import system modules
import arcpy
from arcpy import env
 
import os
try:
    # Set environment settings
    env.workspace = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/OD_Matrix.gdb"
    # Set local variables
    inFeatures = "ODLines3"
    outStatsTable = "CentroidCoverPHC"
    statsFields = [["E2FCA_OD_PHCSlow","sum"],["E2FCA_OD_PHCQuick","sum"]]
    case_field = "OriginID"
    # Execute Statistics to get the area of each vegetation type within the clipped buffer.
    arcpy.Statistics_analysis(inFeatures, outStatsTable, statsFields, case_field)

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [ ]:
# Hos2

In [5]:
# Import system modules
import arcpy
from arcpy import env
 
import os
try:
    # Set environment settings
    env.workspace = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/OD_Matrix.gdb"
    # Set local variables
    inFeatures = "ODLines2"
    outStatsTable = "CentroidCoverHos2"
    statsFields = [["E2FCA_OD_Hos2Slow","sum"],["E2FCA_OD_Hos2Quick","sum"]]
    case_field = "OriginID"
    # Execute Statistics to get the area of each vegetation type within the clipped buffer.
    arcpy.Statistics_analysis(inFeatures, outStatsTable, statsFields, case_field)

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [ ]:
# Hos3

In [6]:
# Import system modules
import arcpy
from arcpy import env
 
import os
try:
    # Set environment settings
    env.workspace = r"C:/Users/kenta/Documents/ArcGIS/Projects/HealthPlaceNew/OD_Matrix.gdb"
    # Set local variables
    inFeatures = "ODLines1"
    outStatsTable = "CentroidCoverHos3"
    statsFields = [["E2FCA_OD_Hos3Slow","sum"],["E2FCA_OD_Hos3Quick","sum"]]
    case_field = "OriginID"
    # Execute Statistics to get the area of each vegetation type within the clipped buffer.
    arcpy.Statistics_analysis(inFeatures, outStatsTable, statsFields, case_field)

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


# Finally, Join all score fields to Census final polygon layer

In [ ]:
# PHC

In [11]:
import arcpy
from arcpy import env
import os
try:
    # Set environment settings
    # env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb"
    
    # Set the local parameters
    inWorkspace1 = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb"
    inWorkspace2 = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\OD_Matrix.gdb"
    inFeatures = os.path.join(inWorkspace2,"Census2015PopChugoku")
    joinField1 = "OBJECTID"
    joinTable = os.path.join(inWorkspace2,"CentroidCoverPHC")
    joinField2 = "OriginID"
    fieldList = ["SUM_E2FCA_OD_PHCSlow", "SUM_E2FCA_OD_PHCQuick"] 
    outFeatures = os.path.join(inWorkspace2,"CensusE2FCAPHC_OD")

    # Join two feature classes by the zonecode field and only carry 
    # over the PropToPopSlow and PropToPopQuick fields
    arcpy.JoinField_management(inFeatures, joinField1, joinTable, joinField2, fieldList)
    arcpy.CopyFeatures_management(inFeatures, outFeatures)

    print("Script completed successfully")
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [ ]:
# Hos2

In [12]:
import arcpy
from arcpy import env
import os
try:
    # Set environment settings
    # env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb"
    
    # Set the local parameters
    inWorkspace1 = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb"
    inWorkspace2 = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\OD_Matrix.gdb"
    inFeatures = os.path.join(inWorkspace2,"CensusE2FCAPHC_OD")
    joinField1 = "OBJECTID"
    joinTable = os.path.join(inWorkspace2,"CentroidCoverHos2")
    joinField2 = "OriginID"
    fieldList = ["SUM_E2FCA_OD_Hos2Slow", "SUM_E2FCA_OD_Hos2Quick"] 
    outFeatures = os.path.join(inWorkspace2,"CensusE2FCAPHCHos2_OD")

    # Join two feature classes by the zonecode field and only carry 
    # over the PropToPopSlow and PropToPopQuick fields
    arcpy.JoinField_management(inFeatures, joinField1, joinTable, joinField2, fieldList)
    arcpy.CopyFeatures_management(inFeatures, outFeatures)

    print("Script completed successfully")
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [ ]:
# Hos3

In [13]:
import arcpy
from arcpy import env
import os
try:
    # Set environment settings
    # env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb"
    
    # Set the local parameters
    inWorkspace1 = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb"
    inWorkspace2 = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\OD_Matrix.gdb"
    inFeatures = os.path.join(inWorkspace2,"CensusE2FCAPHCHos2_OD")
    joinField1 = "OBJECTID"
    joinTable = os.path.join(inWorkspace2,"CentroidCoverHos3")
    joinField2 = "OriginID"
    fieldList = ["SUM_E2FCA_OD_Hos3Slow", "SUM_E2FCA_OD_Hos3Quick"] 
    outFeatures = os.path.join(inWorkspace2,"CensusE2FCAPHCHos2Hos3_OD")

    # Join two feature classes by the zonecode field and only carry 
    # over the PropToPopSlow and PropToPopQuick fields
    arcpy.JoinField_management(inFeatures, joinField1, joinTable, joinField2, fieldList)
    arcpy.CopyFeatures_management(inFeatures, outFeatures)

    print("Script completed successfully")
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [ ]:
# Filter out Shimane pref
# Completed by manually
# New dataset name: CensusE2FCAPHCHos2Hos3_ODShimane

In [17]:
# Import system modules
import arcpy
from arcpy import env
import os
try:
    # Set the workspace
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\OD_Matrix.gdb"
    # Make a layer from the feature class
    arcpy.MakeFeatureLayer_management("CensusE2FCAPHCHos2Hos3_OD", "lyr3") 
    
    # Within selected features, further select only those cities which have a population > 10,000   
    arcpy.SelectLayerByAttribute_management("lyr3", "NEW_SELECTION", "KEN_NAME = '島根県'")
    
    # Write the selected features to a new featureclass
    arcpy.CopyFeatures_management("lyr3", "CensusE2FCAPHCHos2Hos3_ODShimane")
    
    print("Script completed successfully")
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [ ]:
# Now manually impute 0...

In [ ]:
# Filling missing values

In [19]:
import arcpy
# Set geoprocessor object property to overwrite existing output, by default
arcpy.env.overwriteOutput = True
# Local variables ...
arcpy.env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\OD_Matrix.gdb"
try:
    # set local variables
    infeature = "CensusE2FCAPHCHos2Hos3_ODShimane2"
    outfeature = "CensusE2FCAMI_OD"
    fields_to_fill = ["SUM_E2FCA_OD_PHCSLOW", "SUM_E2FCA_OD_PHCQUICK", "SUM_E2FCA_OD_HOS2SLOW", "SUM_E2FCA_OD_HOS2QUICK", "SUM_E2FCA_OD_HOS3SLOW", "SUM_E2FCA_OD_HOS3QUICK"] 
    fill_method = "AVERAGE"
    spatialRef = "CONTIGUITY_EDGES_CORNERS"
    # Fill missing values in a feature class 
    arcpy.FillMissingValues_stpm(infeature, outfeature, fields_to_fill, fill_method, spatialRef)
    
    print("Script completed successfully")
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [ ]:
# Join imputed data's fields to CensusE2FCAFinal data and create a new feature "CensusE2FCAFinalMI"

In [22]:
import arcpy
from arcpy import env
import os
try:
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\OD_Matrix.gdb"
    
    # Set the local parameters
    inWorkspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\OD_Matrix.gdb"
    outWorkspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\OD_Matrix.gdb"
    inFeatures = os.path.join(inWorkspace,"CensusE2FCAPHCHos2Hos3_ODShimane2")
    joinField = "OBJECTID"
    joinTable = os.path.join(inWorkspace,"CensusE2FCAMI_OD")
    fieldList = ["SUM_E2FCA_OD_PHCSLOW", "SUM_E2FCA_OD_PHCQUICK", "SUM_E2FCA_OD_HOS2SLOW", "SUM_E2FCA_OD_HOS2QUICK", "SUM_E2FCA_OD_HOS3SLOW", "SUM_E2FCA_OD_HOS3QUICK"] 
    outFeatures = os.path.join(outWorkspace,"CensusE2FCAFinalMI_OD2")
    
    # Join two feature classes by the zonecode field and only carry 
    # over the PropToPopSlow and PropToPopQuick fields
    arcpy.JoinField_management(inFeatures, joinField, joinTable, joinField, fieldList)
    arcpy.CopyFeatures_management(inFeatures, outFeatures)

    print("Script completed successfully")
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [ ]:
# Delete unnecessary fields

In [24]:
import arcpy
from arcpy import env
try:
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\OD_Matrix.gdb"
    # list all field name
    field_names = [f.name for f in arcpy.ListFields("CensusE2FCAFinalMI_OD2")]
    print(field_names)
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

['OBJECTID', 'Shape', 'KEN_NAME', 'SHICHO_NAME', 'SHI_NAME', 'KU_NAME', 'GUN_NAME', 'CHOSON_NAME', 'OAZA_NAME', 'KOAZA_NAME', 'SUM_AREA', 'TOTPOP_H27', 'SUM_E2FCA_OD_PHCSlow', 'SUM_E2FCA_OD_PHCQuick', 'SUM_E2FCA_OD_Hos2Slow', 'SUM_E2FCA_OD_Hos2Quick', 'SUM_E2FCA_OD_Hos3Slow', 'SUM_E2FCA_OD_Hos3Quick', 'SUM_E2FCA_OD_PHCSLOW_1', 'SUM_E2FCA_OD_PHCQUICK_1', 'SUM_E2FCA_OD_HOS2SLOW_1', 'SUM_E2FCA_OD_HOS2QUICK_1', 'SUM_E2FCA_OD_HOS3SLOW_1', 'SUM_E2FCA_OD_HOS3QUICK_1', 'Shape_Length', 'Shape_Area']


In [25]:
import arcpy
from arcpy import env
try:    
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\OD_Matrix.gdb"
    # Choose the fields want to be deleted
    
    delfield = ['KEN_NAME', 'SHICHO_NAME', 'SHI_NAME', 'KU_NAME', 'GUN_NAME', 'CHOSON_NAME', 'OAZA_NAME', 'KOAZA_NAME', 'SUM_AREA', 'TOTPOP_H27', 'SUM_E2FCA_OD_PHCSlow', 'SUM_E2FCA_OD_PHCQuick', 'SUM_E2FCA_OD_Hos2Slow', 'SUM_E2FCA_OD_Hos2Quick', 'SUM_E2FCA_OD_Hos3Slow', 'SUM_E2FCA_OD_Hos3Quick']   # Delete fields
    arcpy.DeleteField_management('CensusE2FCAFinalMI_OD2',delfield)
    
    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


# Join fields of OD_Matrix E2FCA score to the CensusFinal data

In [ ]:
# First delete fields mistakenly joined with CensusFinal data

In [26]:
import arcpy
from arcpy import env
try:
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb"
    # list all field name
    field_names = [f.name for f in arcpy.ListFields("CensusE2FCAFinalMICent2")]
    print(field_names)
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

['OBJECTID', 'Shape', 'TARGET_FID', 'KEN_NAME', 'SHICHO_NAME', 'SHI_NAME', 'KU_NAME', 'GUN_NAME', 'CHOSON_NAME', 'OAZA_NAME', 'KOAZA_NAME', 'SUM_AREA', 'TOTPOP_H27', 'AreaRes', 'PopDen', 'SUM_PHCE2FCASLOW_12', 'SUM_PHCE2FCAQUICK_12', 'SUM_HOS2E2FCASLOW_12', 'SUM_HOS2E2FCAQUICK_12', 'SUM_HOS3E2FCASLOW_12', 'SUM_HOS3E2FCAQUICK_12', 'Shape_Length', 'Shape_Area']


In [ ]:
# Join fields

In [28]:
import arcpy
from arcpy import env
import os
try:
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\OD_Matrix.gdb"
    
    # Set the local parameters
    inWorkspace1 = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCAStep2Rural.gdb"
    inWorkspace2 = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\OD_Matrix.gdb"
    outWorkspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\OD_Matrix.gdb"
    inFeatures = os.path.join(inWorkspace1,"CensusE2FCAFinalMICent2")
    joinField = "OBJECTID"
    joinTable = os.path.join(inWorkspace2,"CensusE2FCAFinalMI_OD2")
    fieldList = ["SUM_E2FCA_OD_PHCSLOW_1", "SUM_E2FCA_OD_PHCQUICK_1", "SUM_E2FCA_OD_HOS2SLOW_1", "SUM_E2FCA_OD_HOS2QUICK_1", "SUM_E2FCA_OD_HOS3SLOW_1", "SUM_E2FCA_OD_HOS3QUICK_1"] 
    outFeatures = os.path.join(outWorkspace,"CensusE2FCAFinalDis_Con")
    
    # Join two feature classes by the zonecode field and only carry 
    # over the PropToPopSlow and PropToPopQuick fields
    arcpy.JoinField_management(inFeatures, joinField, joinTable, joinField, fieldList)
    arcpy.CopyFeatures_management(inFeatures, outFeatures)

    print("Script completed successfully")
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


# Spatially join with dfnew Geocoded data

In [32]:
### Import system modules
import arcpy
import os

try:
    
    # Set local variables
    inWorkspace1 = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCA.gdb"
    inWorkspace2 = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\OD_Matrix.gdb"
    outWorkdspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\OD_Matrix.gdb"
    targetFeatures = os.path.join(inWorkspace1, "dfnew6Gecoded")
    joinFeatures = os.path.join(inWorkspace2, "CensusE2FCAFinalDis_Con")
    
    # Output will be the target features: PHCCover Polygon with a sum of population field 
    outfc = os.path.join(outWorkdspace, "dfnew6E2FCAFinal_DisCon")
    
    # Create a new fieldmappings and add the two input feature classes.
    fieldmappings = arcpy.FieldMappings()
    fieldmappings.addTable(targetFeatures)
    fieldmappings.addTable(joinFeatures)
       
    # Delete fields that are no longer applicable, such as city CITY_NAME and CITY_FIPS
    # as only the first value will be used by default
    x = fieldmappings.findFieldMapIndex("Status")
    fieldmappings.removeFieldMap(x)
    y = fieldmappings.findFieldMapIndex("Score")
    fieldmappings.removeFieldMap(y)
    z = fieldmappings.findFieldMapIndex("Match_type")
    fieldmappings.removeFieldMap(z)
    b = fieldmappings.findFieldMapIndex("Year")
    fieldmappings.removeFieldMap(b)
    c = fieldmappings.findFieldMapIndex("Pref")
    fieldmappings.removeFieldMap(c)
    d = fieldmappings.findFieldMapIndex("Gun")
    fieldmappings.removeFieldMap(d)
    e = fieldmappings.findFieldMapIndex("City")
    fieldmappings.removeFieldMap(e)
    f = fieldmappings.findFieldMapIndex("Ku")
    fieldmappings.removeFieldMap(f)
    h = fieldmappings.findFieldMapIndex("Match_code")
    fieldmappings.removeFieldMap(h)
    i = fieldmappings.findFieldMapIndex("Addr_type")
    fieldmappings.removeFieldMap(i)
    j = fieldmappings.findFieldMapIndex("In_Single_Line_Input")
    fieldmappings.removeFieldMap(j)
    k = fieldmappings.findFieldMapIndex("TARGET_FID")
    fieldmappings.removeFieldMap(k)
    
    #Run the Spatial Join tool, using the defaults for the join operation and join type
    arcpy.SpatialJoin_analysis(targetFeatures, joinFeatures, outfc, "JOIN_ONE_TO_ONE", "KEEP_ALL", fieldmappings,"COMPLETELY_WITHIN")

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [2]:
# Import system modules
import arcpy
from arcpy import env

try:
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\OD_Matrix.gdb"
    # Set local variables
    inTable = "dfnew6E2FCAFinal_DisCon"
    outLocation = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\OD_Matrix.gdb"
    outTable = "dfnew6E2CAFinal_DisConTable"
    
    # Execute TableToTable
    arcpy.TableToTable_conversion(inTable, outLocation, outTable)

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully


In [3]:
# Import system modules
import arcpy
from arcpy import env

try:
    # Set environment settings
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\OD_Matrix.gdb"
    # Set local variables
    inTable = "dfnew6E2CAFinal_DisConTable"
    outLocation = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\OD_Matrix.gdb"
    out_xls = r"G:\dfnew6E2CAFinal_DisCon.xls"
    
    arcpy.TableToExcel_conversion(inTable, out_xls)

    print("Script completed successfully")
    
except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully
